In [4]:
import os
import pickle
import datetime
import numpy as np 
import pandas as pd

def standarize_column_strings(df, columns, excluded_punctuation=".,*¿?¡!"):
    for col in columns:
        df[col] = df[col].str.lower().astype(str).str.replace(" ", "_")
        df[col] = df[col].str.lower().astype(str).str.replace("-", "_")
        df[col] = df[col].str.lower().astype(str).str.replace("á", "a")
        df[col] = df[col].str.lower().astype(str).str.replace("é", "e")
        df[col] = df[col].str.lower().astype(str).str.replace("í", "i")
        df[col] = df[col].str.lower().astype(str).str.replace("ó", "o")
        df[col] = df[col].str.lower().astype(str).str.replace("ú", "u")
        df[col] = df[col].str.lower().astype(str).str.replace("ü", "u")
        df[col] = df[col].str.lower().astype(str).str.replace(r"[^a-zA-Z\d\_]+", "")
        for ch in excluded_punctuation:
            df[col] = df[col].str.replace(ch, "")

def cleaning(df):
    '''
    Función que convierte las columnas del Data Frame al tipo y forma que se necesita para
    los análisis posteriores
    
    inputs: Data Frame almacenado en el S3 (ingesta.pkl)
    outputs: Data Frame con las variables en formato adecuado (df_clean.pkl)
        
    '''
    #df = pickle.load(open("ingest.pkl","rb"))
    nrows_prev = df.shape[0]
    ncols_prev = df.shape[1]
    data_null_prev = df.isnull().sum().sum()
    # Variables de texto
    df['violations']= df['violations'].astype('object')
    df['violations_count'] = df.violations.str.count(r'\|')+1
    df['violations_count'] = df.violations_count.fillna(0)
    df['violations_count'] = df['violations_count'].astype('int')
    # Variables categóricas
    df['dba_name']= df['dba_name'].astype('object')
    df['aka_name']= df['aka_name'].astype('object')
    df['facility_type']= df['facility_type'].astype('category')
    df['risk']= df['risk'].astype('category')
    df['address']= df['address'].astype('category')
    df['city']= df['city'].astype('category')
    df['state']= df['state'].astype('category')
    df['inspection_type']= df['inspection_type'].astype('category')
    df['results']= df['results'].astype('category')
    # Variable label_risk
    df['risk'] = df['risk'].replace(["Risk 1 (High)"],3)
    df['risk'] = df['risk'].replace(["Risk 2 (Medium)"],2)
    df['risk'] = df['risk'].replace(["Risk 3 (Low)"],1)
    df['risk'] = df['risk'].replace(["All"],0)
    df['risk'] = pd.to_numeric(df['risk'], errors='coerce')
    df=df.rename(columns = {'risk':'label_risk'})
    df['label_risk'] = df['label_risk'].fillna(3)
    df['label_risk'] = df['label_risk'].astype('int')
    # Variables de fecha
    df['inspection_date'] = pd.to_datetime(df['inspection_date'], infer_datetime_format=True)
    df['inspection_month']=df['inspection_date'].dt.month
    MONTH = 12
    df['sin_mnth'] = np.sin(2*np.pi*df.inspection_month/MONTH)
    df['cos_mnth'] = np.cos(2*np.pi*df.inspection_month/MONTH)
    df['inspection_weekday']=df['inspection_date'].dt.weekday
    WEEKDAY = 7
    df['sin_wkd'] = np.sin(2*np.pi*df.inspection_weekday/WEEKDAY)
    df['cos_wkd'] = np.cos(2*np.pi*df.inspection_weekday/WEEKDAY)
    # Etiqueta
    df['label_results'] = df['results'].apply(lambda x: int(0) if x == 'Fail' else (int(1) if x in ['Pass','Pass w/Conditions'] else int(2)))
    df.rename(columns={'license_':'license'}, inplace=True)
    # Imputación de datos
    df.drop(['violations'],axis = 1, inplace = True)
    df.drop(['results'], axis = 1, inplace = True)
    df.drop(df.loc[df['license'].isnull()].index, inplace=True)
    df.drop(df.loc[df['zip'].isnull()].index, inplace=True)
    df.drop(df.loc[df['label_results'] == 2].index, inplace=True)
    df['aka_name'] = df['aka_name'].fillna(df['dba_name'])
    df['dba_name']= df['dba_name'].astype(str).str.lower()
    df['aka_name']= df['aka_name'].astype(str).str.lower()
    df['facility_type']= df['facility_type'].astype(str).str.lower()
    df['state']= df['state'].astype(str).str.lower()
    df['inspection_type']= df['inspection_type'].astype(str).str.lower()
    df = df[~df['state'].isin(['wi', 'ny', 'in'])]
    col_text = ['dba_name','aka_name']
    standarize_column_strings(df, col_text)
    df.rename(columns={'license_':'license'}, inplace=True)
    df_dict_dummy = pd.DataFrame(df['aka_name'])
    df_dict_dummy['facility_type'] = df['facility_type']
    df_dict_dummy.drop(df_dict_dummy.loc[df_dict_dummy['facility_type'].isnull()].index, inplace=True)
    group = df_dict_dummy.groupby('aka_name')
    df_dict_dummy2 = group.apply(lambda x: x['facility_type'].unique())
    df_dict_dummy3 = df_dict_dummy2.to_frame()
    df_dict_dummy3.reset_index(level = 'aka_name', inplace = True)
    df_dict_dummy3 = df_dict_dummy3.rename(columns = {0:'facility_type'})
    df_dict_dummy3['facility_type'] = df_dict_dummy3['facility_type'].apply(lambda x: str(x[0]))
    df2 = pd.merge(df,df_dict_dummy3, how = 'left', on = 'aka_name')
    df2['facility_type_x'] = df2['facility_type_x'].fillna(df2['facility_type_y'])
    df2['facility_type_x'] = df2['facility_type_x'].fillna('restaurant')
    df2=df2.rename(columns = {'facility_type_x':'facility_type'})
    df2.drop(['inspection_id','dba_name','address','city','state','latitude','longitude','location','facility_type_y','inspection_weekday','inspection_month'],axis = 1, inplace = True)
    #pickle.dump(df2,open("df_clean.pkl","wb"))
    nrows_after = df2.shape[0]
    ncols_after = df2.shape[1]
    return df2, nrows_prev, ncols_prev, nrows_after, ncols_after, data_null_prev

In [5]:
df = pd.DataFrame(pickle.load(open("ingest.pkl","rb")))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   inspection_id    1000 non-null   object
 1   dba_name         1000 non-null   object
 2   aka_name         994 non-null    object
 3   license_         1000 non-null   object
 4   facility_type    985 non-null    object
 5   risk             1000 non-null   object
 6   address          1000 non-null   object
 7   city             997 non-null    object
 8   state            1000 non-null   object
 9   zip              997 non-null    object
 10  inspection_date  1000 non-null   object
 11  inspection_type  1000 non-null   object
 12  results          1000 non-null   object
 13  latitude         992 non-null    object
 14  longitude        992 non-null    object
 15  location         992 non-null    object
 16  violations       721 non-null    object
dtypes: object(17)
memory usage: 132.9+

In [6]:
df2, nrows_prev, ncols_prev, nrows_after, ncols_after, data_null_prev = cleaning(df)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 609
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   aka_name          610 non-null    object        
 1   license           610 non-null    object        
 2   facility_type     610 non-null    object        
 3   label_risk        610 non-null    int64         
 4   zip               610 non-null    object        
 5   inspection_date   610 non-null    datetime64[ns]
 6   inspection_type   610 non-null    object        
 7   violations_count  610 non-null    int64         
 8   sin_mnth          610 non-null    float64       
 9   cos_mnth          610 non-null    float64       
 10  sin_wkd           610 non-null    float64       
 11  cos_wkd           610 non-null    float64       
 12  label_results     610 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 66.7+ KB


In [9]:
df2.head(10)

,aka_name,license,facility_type,label_risk,zip,inspection_date,inspection_type,violations_count,sin_mnth,cos_mnth,sin_wkd,cos_wkd,label_results
0,slice_factory,2583193,restaurant,3,60632,2021-03-03,canvass re-inspection,0,1.0,6.123234e-17,0.974928,-0.222521,1
1,five_guys,2617086,restaurant,2,60601,2021-03-03,canvass,2,1.0,6.123234e-17,0.974928,-0.222521,1
2,mangis,20717,restaurant,3,60613,2021-03-03,canvass,2,1.0,6.123234e-17,0.974928,-0.222521,1
3,senor_pan_cafe,2658639,restaurant,3,60639,2021-03-03,canvass re-inspection,9,1.0,6.123234e-17,0.974928,-0.222521,1
4,yolk,2516510,restaurant,3,60654,2021-03-03,canvass,5,1.0,6.123234e-17,0.974928,-0.222521,0
5,jojos_milk_bar,2621841,restaurant,3,60654,2021-03-03,complaint,8,1.0,6.123234e-17,0.974928,-0.222521,1
6,the_great_american_bagelt1_b_14,1879166,restaurant,3,60666,2021-03-03,canvass,2,1.0,6.123234e-17,0.974928,-0.222521,1
7,morgan_park_academy,2142649,school,3,60643,2021-03-03,canvass re-inspection,6,1.0,6.123234e-17,0.974928,-0.222521,0
8,papa_johns_pizza,2069775,restaurant,2,60607,2021-03-03,canvass,4,1.0,6.123234e-17,0.974928,-0.222521,0
9,the_great_american_bagelt3_k2,1879167,restaurant,3,60666,2021-03-03,canvass,3,1.0,6.123234e-17,0.974928,-0.222521,1


In [6]:
col_text = ['dba_name','aka_name']
standarize_column_strings(df, col_text)


NameError: name 'col_text' is not defined

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 609
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   aka_name          610 non-null    object        
 1   license_          610 non-null    object        
 2   facility_type     610 non-null    object        
 3   label_risk        610 non-null    int64         
 4   zip               610 non-null    object        
 5   inspection_date   610 non-null    datetime64[ns]
 6   inspection_type   610 non-null    object        
 7   violations_count  610 non-null    int64         
 8   sin_mnth          610 non-null    float64       
 9   cos_mnth          610 non-null    float64       
 10  sin_wkd           610 non-null    float64       
 11  cos_wkd           610 non-null    float64       
 12  label_results     610 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 66.7+ KB


In [3]:
df2, nrows_prev, ncols_prev, nrows_after, ncols_after, data_null_prev = cleaning(df)

In [16]:
df['label_results'] = df['results'].apply(lambda x: int(0) if x == 'Fail' else (int(1) if x in ['Pass','Pass w/Conditions'] else int(2)))

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   inspection_id     1000 non-null   object  
 1   dba_name          1000 non-null   object  
 2   aka_name          994 non-null    object  
 3   license_          1000 non-null   object  
 4   facility_type     985 non-null    category
 5   risk              1000 non-null   int64   
 6   address           1000 non-null   category
 7   city              997 non-null    category
 8   state             1000 non-null   category
 9   zip               997 non-null    object  
 10  inspection_date   1000 non-null   object  
 11  inspection_type   1000 non-null   category
 12  results           1000 non-null   category
 13  latitude          992 non-null    object  
 14  longitude         992 non-null    object  
 15  location          992 non-null    object  
 16  violations        721 non

In [18]:
df['label_results']

0      1
1      2
2      1
3      2
4      1
      ..
995    1
996    1
997    0
998    1
999    2
Name: label_results, Length: 1000, dtype: int64

In [11]:
df['results'].value_counts()

Pass                  427
Pass w/ Conditions    217
Fail                  185
No Entry               73
Out of Business        70
Not Ready              28
Name: results, dtype: int64

In [12]:
df['license_'].isnull().sum()

0

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   inspection_id     1000 non-null   object  
 1   dba_name          1000 non-null   object  
 2   aka_name          994 non-null    object  
 3   license_          1000 non-null   object  
 4   facility_type     985 non-null    category
 5   risk              1000 non-null   int64   
 6   address           1000 non-null   category
 7   city              997 non-null    category
 8   state             1000 non-null   category
 9   zip               997 non-null    object  
 10  inspection_date   1000 non-null   object  
 11  inspection_type   1000 non-null   category
 12  results           1000 non-null   category
 13  latitude          992 non-null    object  
 14  longitude         992 non-null    object  
 15  location          992 non-null    object  
 16  violations        721 non

In [11]:
pickle.dump(df2,open("clean.pkl","wb"))